1) (5 points) Divide the data set into test/train sets (20/80)

In [1]:
import autograd.numpy as np
from autograd import grad 
import matplotlib.pyplot as plt

In [24]:
# load in dataset
import csv
fname = "hw4_test.csv"
with open(fname) as f:
    reader = csv.reader(f, delimiter=",")

    data = []
    for row in reader:
        data.append(row)

header = data[0]
data = data[1:]

In [25]:
xs = [list(map(float, x[0:9])) for x in data]
ys = [int(x[9]) for x in data]

In [21]:
from sklearn.model_selection import train_test_split 
x_train, x_test, y_train, y_test = train_test_split(xs, ys, train_size=.8, test_size=.2)

2. Implement a multinomial naive bayes classifier **from scratch** with smoothing. (set default smoothing value to 1)

In [5]:
from copy import deepcopy

class MultinomialNaiveBayes():
    def __init__(self, xs, ys, classes, smoothing = 1):
        self.xs = xs
        self.ys = ys
        self.classes = classes
        self.class_counts = [0] * len(self.classes)
        self.smoothing = smoothing
        for y in ys:
            self.class_counts[self.classes.index(y)] += 1
    

    def predict(self, x):
        counts = [deepcopy([self.smoothing] * len(x)) for i in range(0, len(self.classes))]
        
        for x_vals, y in zip(self.xs, self.ys):
            i = self.classes.index(y)
            for j, x_v in enumerate(x_vals):
                if x_v == x[j]:
                    counts[i][j] += 1.0        
        joint = []

        for c in range(0, len(counts)):
            tot = 1
            for i in range(0, len(counts[c])):
                counts[c][i] /= (self.class_counts[c] + len(self.classes))
                tot *= counts[c][i]
            joint.append(tot)

        return self.classes[np.argmax(joint)]

3. Implement a Gaussian naive bayes classifier **from scratch** no smoothing.

In [32]:
from copy import deepcopy
from scipy.stats import norm

class GaussianNaiveBayes():
    def __init__(self, xs, ys, classes):
        self.sds = []
        self.means = []
        self.classes = classes
        filtered = [deepcopy([]) for i in range(0, len(classes))]
        for x, y in zip(xs, ys):
            i = self.classes.index(y)
            filtered[i].append(x)

        

        for i in range(0, len(classes)):
            sds = np.std(filtered[i], axis=0)
            means = np.mean(filtered[i], axis=0)
            
            self.sds.append(sds ** 2)
            self.means.append(means)


    def predict(self, x):
        joint = [1] * len(self.classes)

        for c in range(0, len(self.classes)):
            for i in range(0, len(x)):
                sd = self.sds[c][i]
                mean = self.means[c][i]
                joint[c] *= norm.pdf(x[i], loc=mean, scale=sd)

        print(joint)

        return self.classes[np.argmax(joint)]



4. Calculate the accuracy and the f1 score of test data using both of your models implemented above.

In [7]:
def evaluate(y_actual,y_pred):
    false_positive, false_negative, true_positive, true_negative = 0,0,0,0

    for y_a, y_p in zip(y_actual, y_pred):
        if y_a == y_p:
            if y_a == 1:
                true_positive+=1
            else:
                true_negative+=1
        else:
            if y_a == 1:
                false_negative+=1
            else:
                false_positive+=1

    return [false_positive, false_negative, true_positive, true_negative]

def acc(matrix):
    [_, _, tp, tn] = matrix
    return (tp + tn) / sum(matrix)

def f1(matrix):
    [fp, fn, tp, tn] = matrix
    prec = tp / (tp + fp)
    rec = tp / (tp + fn)

    return 2 * (prec * rec) / (prec + rec)


In [33]:
multi = MultinomialNaiveBayes(x_train, y_train, classes=[2,4])
y_pred = [multi.predict(x) for x in x_test]

matrix = evaluate(y_test, y_pred)
print(f'Accuracy under NaiveBayes: {acc(matrix)}')
print(f'F1 Score under NaiveBayes: {f1(matrix)}')

Accuracy under NaiveBayes: 0.6666666666666666


ZeroDivisionError: division by zero

In [34]:

gauss = GaussianNaiveBayes(xs, ys, classes=[2,4])
y_pred = [gauss.predict(x) for x in x_test]

matrix = evaluate(y_test, y_pred)
print(f'Accuracy under GaussBayes: {acc(matrix)}')
print(f'F1 Score under GaussBayes: {f1(matrix)}')

[6.898370805258773e-17, 7.255735386213203e-11]
[2.1301577364513676e-10, 6.250181424205673e-11]
[2.618574747779024e-09, 1.3775489279542392e-14]
Accuracy under GaussBayes: 1.0


ZeroDivisionError: division by zero